double check against <https://github.com/leelabcnbc/tang_jcompneuro/blob/master/results_ipynb/debug/gabor_debug/gabor_fitting_debug_multi_one_neuron.ipynb>

In [1]:
from tang_jcompneuro.io import load_image_dataset, load_neural_dataset

In [2]:
neuron_to_work_on = 33
y = load_neural_dataset('MkA_Shape')[:, neuron_to_work_on:neuron_to_work_on+1]
X = load_image_dataset('Shape_9500', trans=True, scale=0.5, subtract_mean=True, legacy_rescale=True)
print(X.min(), X.max(), X.mean())
X_mean = X.reshape(len(X), -1).mean(axis=1)
print(X_mean.shape, X_mean.min(), X_mean.max())

-0.85 0.983125 -7.86739094924e-19
(9500,) -1.7763568394e-16 2.48689957516e-16


In [3]:
datasets = (X, y, None, None, None, None)

In [4]:
# get trainer.
from tang_jcompneuro.model_fitting_gabor import get_trainer

In [5]:
trainer = get_trainer('multi,1,1')
results = trainer(datasets)

multi {'num_simple': 1, 'num_complex': 1}
bad index 0/128
bad index 0/128


In [6]:
# pretty match what I got before last cell in previous notebook.
# difference might be due to CUDA version, PyTorch internal difference, etc.
results

{'attrs': {'corr_train': 0.6240257},
 'corr': 0.62402576,
 'model': functools.partial(<function save_model_to_hdf5_group at 0x2b39dc138598>, saved_params={'output_b': -0.01599367, 'complex': {'loc': array([[ 0.04148952],
        [-0.21626778]], dtype=float32), 'sigma': array([[ 0.19938755],
        [ 0.19072005]], dtype=float32), 'orientation': array([ 0.82106137], dtype=float32), 'frequency': array([ 2.60349417], dtype=float32), 'output_a': array([ 0.0047269], dtype=float32)}, 'simple': {'loc': array([[ 0.07270939],
        [-0.10084875]], dtype=float32), 'sigma': array([[ 0.19451664],
        [ 0.11972045]], dtype=float32), 'orientation': array([ 0.97761887], dtype=float32), 'frequency': array([ 1.68466985], dtype=float32), 'output_a': array([ 0.00950284], dtype=float32), 'phase': array([ 4.57942057], dtype=float32), 'bias': array([ 0.82199126], dtype=float32)}}),
 'y_test_hat': array([[-0.01516442, -0.01516442, -0.01516442, ..., -0.00878793,
          0.23867159, -0.00480555]], dtyp

In [7]:
assert callable(results['model'])

In [8]:
# see if we can save models.
import h5py
from tempfile import NamedTemporaryFile
from tang_jcompneuro.gabor import load_model_from_hdf5_group

with NamedTemporaryFile() as f:
    with h5py.File(f.name) as f_tmp:
        results['model'](f_tmp)
        # probably I should not go out of this second with, as that may close f.
        # to avoid all these issues, best way would be create a temp dir, rather than a temp file.
        # but whatever.
        best_params_complex_OUT = load_model_from_hdf5_group(f_tmp)
print(best_params_complex_OUT)

{'complex': {'frequency': array([ 2.60349417], dtype=float32), 'loc': array([[ 0.04148952],
       [-0.21626778]], dtype=float32), 'orientation': array([ 0.82106137], dtype=float32), 'output_a': array([ 0.0047269], dtype=float32), 'sigma': array([[ 0.19938755],
       [ 0.19072005]], dtype=float32)}, 'output_b': -0.01599367, 'simple': {'bias': array([ 0.82199126], dtype=float32), 'frequency': array([ 1.68466985], dtype=float32), 'loc': array([[ 0.07270939],
       [-0.10084875]], dtype=float32), 'orientation': array([ 0.97761887], dtype=float32), 'output_a': array([ 0.00950284], dtype=float32), 'phase': array([ 4.57942057], dtype=float32), 'sigma': array([[ 0.19451664],
       [ 0.11972045]], dtype=float32)}}
